### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
from scripts.AtualizaUltimosJogos import *
import warnings
warnings.filterwarnings('ignore')

### Pegando os jogos do dia

In [2]:
# Obter a data de hoje
hoje = date.today()
ontem = hoje - timedelta(days=1)

### Atualizando os datasets que tem jogo hoje

In [3]:
dia = hoje

In [4]:
# Pegando os jogos do dia para atualizar
jogos_passados = pd.read_csv(f'jogos_do_dia/{dia}.csv')
dia_jogos = jogos_passados['Date'].iloc[0]
id_jogos = jogos_passados['Fixture ID'].unique().tolist()
print(f'Jogos do dia {dia_jogos}\n')

# Buscando os resultados
results = atualiza_ultimos_jogos(id_jogos = id_jogos)

# Merjando os resultados no dataset de jogos do dia
df_resultados = pd.merge(left=jogos_passados, right=results[['Fixture ID', 'Home_Pts', 'Away_Pts', 'Status']], on='Fixture ID', how='left')
df_resultados['Status'].replace(np.nan, 'Não iniciado', inplace=True)

# Organizando as colunas do df_resultados
df_resultados = df_resultados[['Fixture ID', 'Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', 'Status']]
df_resultados = df_resultados.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
df_resultados.sort_values('Time', inplace=True)
df_resultados.reset_index(inplace=True, drop=True)

Jogos do dia 2023-10-24



The msedgedriver version (117.0.2045.31) detected in PATH at c:\Users\johnn\OneDrive\Documentos\apostas\basketball_models\msedgedriver.exe might not be compatible with the detected MicrosoftEdge version (118.0.2088.61); currently, msedgedriver 118.0.2088.61 is recommended for MicrosoftEdge 118.*, so it is advised to delete the driver in PATH and retry
100%|██████████| 18/18 [00:15<00:00,  1.16it/s]


In [5]:
# Criando df com jogos que ainda não terminaram
df_restantes = df_resultados[(df_resultados['Status'] != 'Após Tempo Extra') & (df_resultados['Status'] != 'Encerrado') & (df_resultados['Status'] != 'Após Pênaltis')]
df_restantes = df_restantes[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Status']]

# Limpando o df_resultados
df_resultados = df_resultados[(df_resultados['Status'] == 'Após Tempo Extra') | (df_resultados['Status'] == 'Encerrado') | (df_resultados['Status'] == 'Após Pênaltis')]
df_resultados = df_resultados[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A']]

# Exclui os registros que já existem no dataset da temporada
temporada = pd.read_csv('data/Entire_Season.csv')
n_antes = len(temporada)
temporada = pd.concat([temporada, df_resultados], ignore_index=True)
temporada = temporada.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
temporada.reset_index(drop=True, inplace=True)
temporada.to_csv('data/Entire_Season.csv', index=False)

print(f'Dataset de temporada atualizado com {len(temporada) - n_antes} jogos.\n')
print('----------------------------------------------------------\n')
print(f'Atenção! {len(df_restantes)} jogos não finalizados.\n')
display(df_restantes)

Dataset de temporada atualizado com 6 jogos.

----------------------------------------------------------

Atenção! 12 jogos não finalizados.



,Date,League,Time,Home,Away,Home_Pts,Away_Pts,Status
6,2023-10-24,Europa Liga Dos Campeões,13:30,Peristeri,Szombathely,81.0,68.0,4º Quarto9'
7,2023-10-24,Europa Eurocopa,14:30,Hamburg,Hapoel Tel-Aviv,42.0,59.0,Intervalo
8,2023-10-24,Europa Eurocopa,15:00,Venezia,Joventut,14.0,12.0,1º Quarto9'
9,2023-10-24,França Copa Da França,15:00,Rouen,Nanterre,3.0,9.0,1º Quarto3'
10,2023-10-24,Europa Liga Dos Campeões,15:00,Ludwigsburg,Szczecin,23.0,27.0,2º Quarto1'
11,2023-10-24,Europa Eurocopa,15:30,London Lions,Paris,NaN,NaN,Não iniciado
12,2023-10-24,Europa Liga Dos Campeões,15:30,Unicaja,Le Mans,NaN,NaN,Não iniciado
13,2023-10-24,Brasil Nbb,20:00,Bauru,São Paulo,NaN,NaN,Não iniciado
14,2023-10-24,Brasil Nbb,20:00,Caxias do Sul,União Corinthians,NaN,NaN,Não iniciado
15,2023-10-24,Brasil Nbb,20:00,Botafogo,Pato,NaN,NaN,Não iniciado
